In [1]:
import openmc
import analysis
import os
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import numpy as np

In [2]:
def plotting_2D(material = "W", tally = 'fast flux', score = 'flux', thickness = '55'):
    # Setting up the plot parameters
    plot_width = 60
    #plot_height = 640
    plot_height = 150
    extent=(-plot_width, plot_width, 0, plot_height)
    sp = openmc.StatePoint("shielding/" + material + "/" + thickness + "/statepoint.10.h5", autolink=False)
    background_image = plt.imread("shielding/" + material + "/" + thickness + "/background.png")
    try:
        os.mkdir("shielding_plots/2D_plots/" + material + "/" + thickness)
    except OSError as error:
        #print(error)
        pass
    analysis.plot_result(sp, background=background_image, tally_name=tally, mesh_dims=(150,30,30), save_aggregate=False, plot_extent=extent, m_factor=1/16, 
                         contour_lvl=np.logspace(-10, -2, 20), clabel="$[n/cm^2-s]$", title='DT ' + tally + '\n ' + thickness + ' cm thick ' + material, nuclide_dim=None,
                         savedir="shielding_plots/2D_plots/" + material + "/" + thickness, tally_score=score)

In [3]:
def plot_target_flux_energy_to_thickness(material_list):
    """
    Plot the fast, epithermal and thermal flux against the thickness of the shield for various materials
    And an annodated heat map that compares the effectiveness of shield material at different thicknesses
    """
    output_dir = "/mnt/d/WHAM_OpenMC/Shielding"
    #material_list = os.listdir(output_dir)
    #material_list.sort()
    # 2D arrays for heatmap
    fast_2d, thermal_2d, epithermal_2d = [], [], []
    for material in material_list:
        fast, fast_err = [], []
        epithermal, epithermal_err = [], []
        thermal, thermal_err = [], []
        thickness = os.listdir(output_dir + "/" + material)
        thickness.sort()
        for t in thickness:
            run_dir = output_dir + "/" + material + "/" + t
            statepoint = openmc.StatePoint(run_dir + "/statepoint.10.h5")
            target_flux_tally = statepoint.get_tally(name = "neutron flux target")
            target_flux_dataframe = target_flux_tally.get_pandas_dataframe(float_format='{:.9e}')
            #print(material)
            #print(target_flux_dataframe)
            target_cell_volume = np.pi*50**2*10 

            fast.append(target_flux_dataframe.at[2, 'mean']/target_cell_volume)
            fast_err.append(target_flux_dataframe.at[2, 'std. dev.']/target_cell_volume)
            epithermal.append(target_flux_dataframe.at[1, 'mean']/target_cell_volume)
            epithermal_err.append(target_flux_dataframe.at[1, 'std. dev.']/target_cell_volume)
            thermal.append(target_flux_dataframe.at[0, 'mean']/target_cell_volume)
            thermal_err.append(target_flux_dataframe.at[0, 'std. dev.']/target_cell_volume)
        fig = plt.figure(figsize=(12, 8))
        plt.plot(thickness, fast, 'b-', label="fast flux (>100 keV)")
        plt.errorbar(thickness, fast, fast_err, fmt='b-')
        plt.plot(thickness, epithermal, 'g-', label="epithermal flux (0.5 eV - 100 keV)")
        plt.errorbar(thickness, epithermal, epithermal_err, fmt='g-')
        plt.plot(thickness, thermal, 'r-', label="thermal flux (<0.5 eV)")
        plt.errorbar(thickness, thermal, thermal_err, fmt='r-')
        plt.title("Neutron Flux Behind Shield\n" + material)
        plt.xlabel("Shield Thickness (cm)")
        plt.yscale('log')
        plt.ylabel("Neutron flux $[n/cm^2-s]$")
        plt.ylim([1e-11, 1e-4])
        plt.legend()
        print("finished plotting flux against thickness for " + material)
        fig.savefig("./shielding_plots/flux against thickness " + material)
        plt.close()

        # annodated heat map for fast, epithermal and thermal flux respectively
        fast_2d.append(fast)
        thermal_2d.append(thermal)
        epithermal_2d.append(epithermal)
        subtitles = ["Fast Flux", "Epithermal Flux", "Thermal Flux"]
        n = 0
    for flux_array in [fast_2d, epithermal_2d, thermal_2d]:
        #print(flux_array)
        
        fig = plt.figure(figsize=(18, 12))
        im = plt.imshow(flux_array, cmap=cm.plasma, norm=colors.LogNorm(vmin=1e-7, vmax=1e2))

        plt.xticks(np.arange(len(thickness)), labels=thickness)
        plt.yticks(np.arange(len(material_list)), labels=material_list)

        # Loop over data dimensions and create text annotations.
        for i in range(len(material_list)):
            for j in range(len(thickness)):
                data = flux_array[i][j]
                text = plt.text(j, i, f"{data:.2e}", size=6,
                            ha="center", va="center", color="black")
        cb = plt.colorbar(im, label = "Neutron flux $[n/cm^2-s]$")
        plt.title("Material Shielding Effectiveness Comparison\n" + subtitles[n] + " (lower is better)")
        plt.xlabel("Thickness (cm)")
        plt.clim([1e-11, 1e-4])
        plt.tight_layout()
        plt.savefig("./shielding_plots/material comparison " + subtitles[n])
        plt.show()
        plt.close()
        n += 1

    return

In [4]:
#material_list = ['W', 'WC', 'WB2', 'WB', 'W2B5', 'ZrH2', 'TiH2', 'Ti_HfH2', 'MgO_HfH2', 'Fe_HfH2_WB2', 'HfH2', 'B4C', 'B4C_epoxy_mix', 'stainless_304', 'H2O']
material_list = ['W2B5']
thickness_list = np.arange(40, 90, 5)
for material in material_list:
    for t in thickness_list:
        plotting_2D(material, 'fast flux', thickness=str(t))
        #plotting_2D(material, 'total neutron flux', thickness=str(t))
        #plotting_2D(material, 'photon flux', thickness=str(t))
        #plotting_2D(material, 'epithermal flux', thickness=str(t))
        #plotting_2D(material, 'thermal flux', thickness=str(t))
        #plotting_2D(material, 'radiative capture', score='(n,gamma)', thickness=str(t))
        #plotting_2D(material, 'neutron heat load', score = 'heating', thickness=str(t))
        #plotting_2D(material, 'neutron local heat load', score = 'heating-local', thickness=str(t))
        plotting_2D(material, 'helium, hydrogen production, (n,alpha)', score = '(n,a)', thickness=str(t))

/home/mason/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/mason/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/d/WHAM_OpenMC/analysis.py:194: UserWarning: Log scale: values of z <= 0 have been masked
  CS = plt.contour(np.multiply(data, m_factor), contour_lvl, origin="lower", norm=colors.LogNorm(vmin=min(contour_lvl), vmax=max(contour_lvl)),
/home/mason/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/mason/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/d/WHAM_OpenMC/analysis.py:194: UserWarning: Log scale: values of z 

In [ ]:
plotting_2D('W2B5', 'fast flux')

In [ ]:
material_list = ['W', 'WC', 'WB2', 'WB', 'W2B5', 'ZrH2', 'TiH2', 'Ti_HfH2', 'MgO_HfH2', 'Fe_HfH2_WB2', 'HfH2', 'B4C', 'B4C_epoxy_mix', 'stainless_304', 'H2O']
plot_target_flux_energy_to_thickness(material_list)